In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math

# Making the Graph

`init_graph` : this function just makes the structure of the graph that will represent the talukas. Also, there was some trouble in the Adjecency file(on the 81th taluka), so I simply created the graph, and removed the 81th index taluka(which wasn't there in the 1st place)


`init_graph_attr` : Here, we are just initializing the graph, with capability vector, taluka name, and stress = 0.

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        if snode == 80:
            continue
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            # removed the if condition of k = 81
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    G.remove_node(80) #removed the extra node made during graph creation
    return

In [3]:
def init_graph_attr(G,AdjFile,df,col1, col2):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["KGISTalukN"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G,node_adj_frame)
    capability_vector = list(zip(df[col1], df[col2]))
    node_attri_dict = dict(zip(df["Taluka"],capability_vector))
    node_attri_dict = dict((k.lower(), v) for k, v in node_attri_dict.items())
    for i in range(len(node_adj_frame)):
        temp = {}
        if i == 80:
            continue
        temp["capabilityvector"] = node_attri_dict[node_list[i].lower()]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp
    # explicitly added the last taluka from the adj. list
    nt = {}
    nt["capabilityvector"] = node_attri_dict[node_list[226].lower()]
    nt["nodeStress"] = 0
    nt["name"] = "Hadagali"
    nodeAttr[226] = nt
    nx.set_node_attributes(G, nodeAttr)

In [4]:
# initializing the graph from networkx library.
G = nx.Graph()
df = pd.read_excel('../input_files/Combined_PIA_IMR_MMR.xlsx')

In [5]:
#this dictionary will be having District and a list of talukas inside them.
dist_taluka_dict = defaultdict(list)
for k, v in zip(df["District_GIS"], df["Taluka"]):
    dist_taluka_dict[k].append(v)

L2 normalization

In [6]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

# Calculating Stress

Here, we are calculating stress. These are the steps:
1. Calculate the centeroid for each node, that will be the average of the neighbors of elements of the capability vector. Now, here, we are taking the centroid as the sum of those values, and not average. We will average them at the time of calculation.
2. Now, take the l2 distance between the capability vector of the node and it's centroid. This will be the stress. Also, note that stability is just 1-stress for the taluka, and that is the only difference between get_node_stress and get_node_stability.

In [7]:
def get_node_stress(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
def get_node_stability(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

Calculating initial stress before any intervention

In [9]:
df['Taluka'] = df['Taluka'].str.lower()
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized MMR", "Normalized IMR")
initialstress = get_node_stress(G)

In [10]:
df["Initial Stress"] = df["Taluka"].map(initialstress)

Stress Calculation after ANC -20%

In [11]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-20%)", "Normalized MMR (ANC -20%)")
ANCminus20stress = get_node_stress(G)
df["Stress(ANC - 20%)"] = df["Taluka"].map(ANCminus20stress)

Stress Calculation after ANC +20%

In [12]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+20%)", "Normalized MMR (ANC +20%)")
ANCplus20stress = get_node_stress(G)
df["Stress(ANC + 20%)"] = df["Taluka"].map(ANCplus20stress)

Stress Calculation after ANC -10%

In [13]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-10%)", "Normalized MMR (ANC -10%)")
ANCminus10stress = get_node_stress(G)
df["Stress(ANC - 10%)"] = df["Taluka"].map(ANCminus10stress)

Stress Calculation after ANC +10%

In [14]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+10%)", "Normalized MMR (ANC +10%)")
ANCplus10stress = get_node_stress(G)
df["Stress(ANC + 10%)"] = df["Taluka"].map(ANCplus10stress)

In [42]:
df["Normalized IMR (ANC+10%)"]

0      0.936246
1      0.854066
2      0.882655
3      0.587826
4      0.621669
         ...   
221    0.641837
222    0.803416
223    0.534068
224    0.712948
225    0.868064
Name: Normalized IMR (ANC+10%), Length: 226, dtype: float64

# Aggregation

The aggregate function simple takes the average of all the talukas in a district and maps them to the corresponding district. This function outputs a dictionary with key: district name and value: value to be aggregated.

In [15]:
def aggregate(taluka_dict):
    dist_stress = {}
    for dist, taluks in dist_taluka_dict.items():
        agg_stress = 0
        for taluk in taluks:
            try:
                agg_stress = agg_stress + taluka_dict[taluk.lower()]
            except(KeyError):
                pass
        dist_stress[dist] = agg_stress/len(taluks)
    return dist_stress

In [16]:
df_imp_IMR = pd.read_excel("../input_files/PIA_Normalized_IMR.xlsx")
df_imp_MMR = pd.read_excel("../input_files/PIA_Normalized_MMR.xlsx")

In [17]:
combined_Impact_ANC = pd.DataFrame() #it will have taluka stress

In [18]:
aggregate_df = pd.DataFrame() #initialized dataframe for district level calculations

# Calculating Stress

In the next 8 steps, I am just calculating the stress values for different values of ANC. Now, It can be put in a for loop, but the column names are not in any pattern that will give an easy for loop. Also, the column names might change in the future for the PIA file. So, that's why I did a little bit of brute forcing.


Here, will will get the taluka level stress in combined_Impact_ANC and district level stress in aggregate_df

In [19]:
# ===================================================FOR ANC -20%==============================================================
# Getting the graph ready for ANC -20%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-20%)", "Normalized MMR (ANC -20%)")

# These dict have the taluka names and Impact Score for ANC -20%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC -20%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact Score (ANC -20%)"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating IMR and MMR impact scores
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress for 
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 20%)',1:'IMPACT_SCORE_MMR (ANC - 20%)', 2:'STRESS_SCORE (ANC - 20%)'}, inplace = True)
aggregate_df = aI_df

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 20%)', 1:'STRESS_SCORE (ANC - 20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC - 20%)', 1:'STRESS_SCORE (ANC - 20%)'}, inplace = True)

In [20]:
combined_Impact_ANC = trdIMR
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC - 20%)'] = trdMMR['IMPACT_SCORE_MMR (ANC - 20%)']

In [21]:
# ===================================================FOR ANC -10%==============================================================
# Getting the graph ready for ANC -10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-10%)", "Normalized MMR (ANC -10%)")

# These dict have the names vs Impact Score for ANC -10%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC -10%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact_____Deprivation % - Households with with any woman has not received at least 4 antenatal care visits for the most recent birth or has not received assistance from trained skilled medical personnel during the most recent childbirth._____-10%"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 10%)',1:'IMPACT_SCORE_MMR (ANC - 10%)', 2:'STRESS_SCORE (ANC -10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (ANC - 10%)'] = aI_df['IMPACT_SCORE_IMR (ANC - 10%)']
aggregate_df['IMPACT_SCORE_MMR (ANC - 10%)'] = aI_df['IMPACT_SCORE_MMR (ANC - 10%)']
aggregate_df['STRESS_SCORE (ANC -10%)'] = aI_df['STRESS_SCORE (ANC -10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 10%)', 1:'STRESS_SCORE (ANC -10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC - 10%)', 1:'STRESS_SCORE (ANC -10%)'}, inplace = True)

In [22]:
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC - 10%)'] = trdMMR['IMPACT_SCORE_MMR (ANC - 10%)']
combined_Impact_ANC['IMPACT_SCORE_IMR (ANC - 10%)'] = trdIMR['IMPACT_SCORE_IMR (ANC - 10%)']
combined_Impact_ANC['STRESS_SCORE (ANC - 10%)'] = trdMMR['STRESS_SCORE (ANC -10%)']

In [23]:
# ===================================================FOR ANC +10%==============================================================
# Getting the graph ready for ANC +10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+10%)", "Normalized MMR (ANC +10%)")

# These dict have the names vs Impact Score for ANC +10%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC +10%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact_____Deprivation % - Households with with any woman has not received at least 4 antenatal care visits for the most recent birth or has not received assistance from trained skilled medical personnel during the most recent childbirth._____+10%"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 10%)',1:'IMPACT_SCORE_MMR (ANC + 10%)', 2:'STRESS_SCORE (ANC +10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (ANC + 10%)'] = aI_df['IMPACT_SCORE_IMR (ANC + 10%)']
aggregate_df['IMPACT_SCORE_MMR (ANC + 10%)'] = aI_df['IMPACT_SCORE_MMR (ANC + 10%)']
aggregate_df['STRESS_SCORE (ANC +10%)'] = aI_df['STRESS_SCORE (ANC +10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 10%)', 1:'STRESS_SCORE (ANC +10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC + 10%)', 1:'STRESS_SCORE (ANC +10%)'}, inplace = True)

In [24]:
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC + 10%)'] = trdMMR['IMPACT_SCORE_MMR (ANC + 10%)']
combined_Impact_ANC['IMPACT_SCORE_IMR (ANC + 10%)'] = trdIMR['IMPACT_SCORE_IMR (ANC + 10%)']
combined_Impact_ANC['STRESS_SCORE (ANC + 10%)'] = trdMMR['STRESS_SCORE (ANC +10%)']

In [25]:
# ===================================================FOR ANC +20%==============================================================
# Getting the graph ready for ANC -10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+20%)", "Normalized MMR (ANC +20%)")

# These dict have the names vs Impact Score for ANC +20%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC +20%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact_____Deprivation % - Households with with any woman has not received at least 4 antenatal care visits for the most recent birth or has not received assistance from trained skilled medical personnel during the most recent childbirth._____+20%"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 20%)',1:'IMPACT_SCORE_MMR (ANC + 20%)', 2:'STRESS_SCORE (ANC +20%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (ANC + 20%)'] = aI_df['IMPACT_SCORE_IMR (ANC + 20%)']
aggregate_df['IMPACT_SCORE_MMR (ANC + 20%)'] = aI_df['IMPACT_SCORE_MMR (ANC + 20%)']
aggregate_df['STRESS_SCORE (ANC +20%)'] = aI_df['STRESS_SCORE (ANC +20%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 20%)', 1:'STRESS_SCORE (ANC +20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC + 20%)', 1:'STRESS_SCORE (ANC +20%)'}, inplace = True)

In [26]:
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC + 20%)'] = trdMMR['IMPACT_SCORE_MMR (ANC + 20%)']
combined_Impact_ANC['IMPACT_SCORE_IMR (ANC + 20%)'] = trdIMR['IMPACT_SCORE_IMR (ANC + 20%)']
combined_Impact_ANC['STRESS_SCORE (ANC + 20%)'] = trdMMR['STRESS_SCORE (ANC +20%)']

In [27]:
combined_Impact_ANC.to_excel('../output_files_2d/2D_talukaLevel_stress_impact.xlsx')
aggregate_df.to_excel('../output_files_2d/2D_districtLevel_stress_impact.xlsx')

# Sustainable Intervention Score Calculations

The formula for calculating the score is:  $$score = (impact_{avg} * stability) \over dissonance $$

`district_SI_score`: it calculates the score of the districts. We pass empty lists in it and this function populates them with values. It uses the aggregated values of stress and impacts.


`taluka_SI_score`: it calculates the score of the talukas. We pass empty lists in it and this function populates them with values. It uses the taluka level values of stress and impacts.

`normailize`: takes a dataframe ,column name, empty list, the upper limit and lower limit of normalization values and returna normalized. Let the normalization range be [a, b], and, X be the column, then, the formula used for normalization is: $$X_{normalized}[i] = a + (b - a) * { (X_{original}[i] - max(X_{original})) \over (max(X_{original}) - mix(X_{original}))}$$

In [28]:
df_score = pd.DataFrame()
df_score['Taluka'] = df['Taluka']

In [29]:
dis_temp = ['+2', '+1', '-1', '- 2']

In [30]:
def district_SI_score(df, score_list, i, diss_list, imr_impact, mmr_impact, avg_impact):
    for j in range(len(df)):
        str_imr = 'IMPACT_SCORE_IMR (ANC ' + i + '0%)'
        str_mmr = 'IMPACT_SCORE_MMR (ANC ' + i + '0%)'
        str_stress = 'STRESS_SCORE (ANC ' + i + '0%)'

        imr_impact.append(df[str_imr][j])
        mmr_impact.append(df[str_mmr][j])
        mean_impact = (df[str_imr][j] + df[str_mmr][j]) / 2 #mean impact of the taluka
        avg_impact.append(mean_impact)
        dissonance = abs(df[str_imr][j] - df[str_mmr][j]) #dissonance between the taluka
        diss_list.append(dissonance)
        score = (mean_impact * (1 - df[str_stress][j])) / dissonance
        score_list.append(score)

In [31]:
def taluka_SI_score(df, score_list, i):
    for j in range(len(df)):
        str_imr = 'IMPACT_SCORE_IMR (ANC ' + i + '0%)'
        str_mmr = 'IMPACT_SCORE_MMR (ANC ' + i + '0%)'
        str_stress = 'STRESS_SCORE (ANC ' + i + '0%)'

        mean_impact = (df[str_imr][j] + df[str_mmr][j]) / 2 #mean impact of the taluka

        dissonance = abs(df[str_imr][j] - df[str_mmr][j]) #dissonance between the taluka

        score = (mean_impact * (1 - df[str_stress][j])) / dissonance
        score_list.append(score)

In [32]:
temp = ['+ 2', '+ 1', '- 1', '- 2'] #for automation of all interventions
for i in temp:
    score_list = []
    taluka_SI_score(combined_Impact_ANC, score_list, i)
    str = 'SI_Score (ANC ' + i + '0%)'
    df_score[str] = score_list

In [33]:
df_score.to_excel('../output_files_2d/2D_talukaLevel_SI_scores.xlsx')

Aggregating the SI_Scores at district level

In [34]:
aggregated_score_df = pd.DataFrame()
aggregated_score_df['District'] = dist_taluka_dict.keys() #adding new district column
# aggregated_score_df

In [35]:
def normalize(df, col_str, nd_list, lower_lim, upper_lim):
    for j in range(len(df)):
        norm_dis = 0
        norm_dis = lower_lim + (upper_lim - lower_lim)*(df[col_str][j] - min(df[col_str]))/(max(df[col_str]) - min(df[col_str]))
        nd_list.append(norm_dis)

In [36]:
aggregate_df.rename(columns={'STRESS_SCORE (ANC +10%)':'STRESS_SCORE (ANC + 10%)'}, inplace=True)
aggregate_df.rename(columns={'STRESS_SCORE (ANC +20%)':'STRESS_SCORE (ANC + 20%)'}, inplace=True)
aggregate_df.rename(columns={'STRESS_SCORE (ANC -10%)':'STRESS_SCORE (ANC - 10%)'}, inplace=True)

In [37]:
for i in temp:
    score_list = []
    diss_list = []
    imr_impact = []
    mmr_impact = []
    avg_impact = []
    district_SI_score(aggregate_df, score_list, i, diss_list, imr_impact, mmr_impact, avg_impact)
    score_str = 'SI_Score (ANC ' + i + '0%)'
    str_imr = 'IMPACT_SCORE_IMR (ANC ' + i + '0%)'
    str_mmr = 'IMPACT_SCORE_MMR (ANC ' + i + '0%)'
    str_diss = 'DISSONANCE (ANC ' + i + '0%)'
    str_avg_imr = 'MEAN IMR (ANC ' + i + '0%)'
    aggregated_score_df[score_str] = score_list
    aggregated_score_df[str_imr] = imr_impact
    aggregated_score_df[str_mmr] = mmr_impact
    aggregated_score_df[str_avg_imr] = avg_impact
    aggregated_score_df[str_diss] = diss_list

In [38]:
#here, we are saving the raw sacores
aggregated_score_df.to_excel('../output_files_2d/2D_districtLevel_SI_scores.xlsx') 

In [39]:
for i in temp:
    normalized_score_list = []
    score_str = 'SI_Score (ANC ' + i + '0%)'
    normalize(aggregated_score_df, score_str, normalized_score_list, 0, 1)
    aggregated_score_df[score_str] = normalized_score_list

In [40]:
#here, we are saving the normalized scores.
aggregated_score_df.to_excel('../output_files_2d/2D_districtLevel_normalized_SI_scores.xlsx')